# binary class and examples

## ADT tree class

In [5]:
class Tree:
    """abstract base class representing a tree structure"""
    #----------nested position class--------------------
    class Position:
        """an abstraction representing the location of a single element"""
        def element(self):
            raise NotImplementedError('must be implemented by subclass')
            
        def __eq__(self,other):
            raise NotImplementedError('must be implemented by subclass')
            
        def __ne__(self,other):
            return not (self == other)
        
    #------------abstrct method-----------------------
    def root(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def parent(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def num_children(self,p):
        raise NotImplementedError('must be implemented by subclass')
    
    def children(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')
        
    # -----------concrete methods implemented in this class
    def is_root(self, p):
        return self.root() == p

    def is_leaf(self,p):
        return self.num_children(p) == 0
    
    def is_empty(self):
        return len(self) == 0
    
    def depth(self,p):
        if self.si_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))
        
    def _height1(self):
        return max(self.depth(p) for p in self.position() if self.is_leaf(p))
    
    def _height2(self,p):
        if self.is_leaf(p):
            return 0
        else:
            return 1+max(self._height2(c) for c in self.children(p))
        
    def height(self, p=None):
        if p is None:
            p = self.root()
        return self._height2(p)
    
    #------------tree iteration methods----------
    def __iter__(self):
        for p in self.position():
            yield p.element
            
    def preorder(self):
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p
                
    def _subtree_preorder(self,p):
        yield p
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other
                
    def position(self):
        return self.preorder()
    
    def postorder(self):
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
                
    def _subtree_postorder(self,p):
        for c in self.children(p):
            for other in self._subtree_postorder(c):
                yield other
        yield p
        
    def breadthfirst(self):
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())
            while not fringe.dequeue():
                p = fringe.dequeue()
                yield p
                for c in self.children(p):
                    fringe.enqueue(c)

## binaryTree class

In [6]:
class BinaryTree(Tree):
    #-additional abstract methods---------
    def left(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def right(self,p):
        raise NotImplementedError('must be implemented by subclass')
        
    def sibling(self,p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self,p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)
            
    #---inorder traverse tree----------
    def inorder(self):
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p
                
    def _subtree_inorder(self,p):
        if self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other
                
    def position(self):
        return self.inorder()

## linkedBinary Tree class

In [4]:
class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'
        def __init__(self, element, parent= None, left=None,right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
            
    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            sefl._node = node
            
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node
        
        def _validate(self,p):
            if not isinstance(p, self.Position):
                raise TypeError('position type error')
            if p._container is not self:
                raise ValueError('p does not belong to this container')
            if p._node._parent is p._node:
                raise ValueError('p is no longer valid')
            return p._node
        
        def _make_position(self, node):
            return self.Position(self, node) if node is not None else None
        
    #----binary tree constructor-----------
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self,p):
        node = sefl._validate(p)
        return self._make_position(self._parent)
    
    def left(self,p):
        node = sefl._validate(p)
        return self._make_position(self._left)
    
    def right(self,p):
        node = sefl._validate(p)
        return self._make_position(self._right)
    
    def num_children(self,p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count
    
    def _delete(self,p):
        node = self._validate(p)
        if self.num_children(p) == 2: raise ValueError('2 children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element
    
    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2): raise TypeError('type match')
        self._size += len(t1) +len(t1)
        if not ti.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0
            
                
            

### traverse application

In [7]:
# preoreder for print chapter contents
def preorder_indent(T,p,d):
    print(2*d*' ' + str(p.element()))
    for c in T.children(p):
        preorder_indent(T,c,d+1)

In [8]:
def preorder_pabel(T,p,d,path):
    label = '.'.join(str(j+1) for j in path)
    print(2*d*' ' +label, p.element())
    path.append(0)
    for c in T.children(p):
        preorder_label(T,c,d+1,path)
        path[-1] +=1
        path.pop()

## eulertour traverse

In [9]:
class EulerTour:
    def __init__(self, tree):
        self._tree = tree
        
    def tree(self):
        return self._tree
    
    def execute(self):
        if len(self._tree) > 0:
            return self._tour(self._tree.root(),0,[])             # start recursion
        
    def _tour(self, p, d, path):# position of current node, depth of node, list of indices of children root to p
        self._hook_previsit(p,d,path)
        result =[]
        path.append(0)
        for c in self._tree.children(p):
            result.append(self._tour(c,d+1,path))
        path.pop()
        answer= self._hook_postvisit(p,d,path,result)
        return answer
    
    def _hook_previsit(self,p,d,path):
        pass
    def _hook_postvisit(self,p,d,path,result):
        pass
            
        

In [10]:
class ParenthesizeTour(EulerTour):
    def _hook_previsit(self, p, d, path):
        if path and path[-1]>0:
            print(', ',end='')
            if not self.tree().is_leaf(p):
                print(' (',end='')
                
    def _hook_postvisit(self,p,d,path,result):
        if not self.tree().is_leaf(p):
            print(')',end='')

In [11]:
class BinaryEulerTour(EulerTour):
    def _tour(self,p,d,path):
        result = [None, None]
        self._hook_previsit(p,d,path)
        if self._tree.left(p) is not None:
            path.append(0)
            result[0] = self._tour(self._tree.left(p),d+1,path)
            path.pop()
        self._hook_invisit(p,d,path)
        if self._tree.right(p) is not None:
            path.append(1)
            result[1] = self._tour(self._tree.right(p),d+1,path)
            path.pop()
        answer = self._hook_postvisit(p,d,path,result)
        return answer
    
    def _hook_invisit(self,p,d,path): pass

In [12]:
class BinaryLayout(BinaryEulerTour):
    def __init__(self,tree):
        super().__init__(tree)
        self._count=0
        
    def _hook_invisit(self,p,d,path):
        p.element().setX(self._count)
        p.element().setY(d)
        self._count += 1